# Setting up Azure
Now that we've testing things locally, we're ready to use Azure to scale up our workload, automate and operationalize the job. 

First, we will walk through setting up credentials and settings for connecting to and creating resources in Azure. Then we will create an Azure blob storage which will be used throughout the tutorial as our object store. 

---

Import utilities.

In [ ]:
import re

## Setting up our credentials
In order to deploy and use resources on Azure, we have to set up a [Service Principal](https://docs.microsoft.com/en-us/cli/azure/create-an-azure-service-principal-azure-cli?toc=%2Fen-us%2Fazure%2Fazure-resource-manager%2Ftoc.json&bc=%2Fen-us%2Fazure%2Fbread%2Ftoc.json&view=azure-cli-latest) to authenticate our deployments. 

We'll also need to make sure we've identified which subscription_id and region to use. For this project, we'll also create a new resource group to contain all the resources that we create.

This section of the notebook will walk through setting up these credentials using the __az cli__.

Use the __az cli__ to generate service principal credentials.

In [ ]:
credentials = !az ad sp create-for-rbac -o json --query "[appId, password, tenant]"

Use regex to pull out the __appId__ (which we'll call __client id__), the __password__ (which we'll call __secret__), and the tenant id.

In [ ]:
[aad_client_id, aad_secret, aad_tenant] = re.findall("\w{8}[-]\w{4}[-]\w{4}[-]\w{4}[-]\w{12}", str(credentials))

Since we'll use these credentials throughout the tutorial, lets same them into our .env file.

In [ ]:
!dotenv set AAD_CLIENT_ID $aad_client_id
!dotenv set AAD_SECRET $aad_secret
!dotenv set AAD_TENANT $aad_tenant

Select which subscription you'd like to use for this project:
1. Print out all available subscriptions associated with your account
2. Set the Subscription ID you'd like to use as variable `subscription_id`
3. In the __az cli__, set the active subscription

In [ ]:
!az account list -o table

In [ ]:
subscription_id = "<your-selected-subscription-id>"

In [ ]:
!az account set -s $subscription_id

Choose a name for the resource group you'll be using for this project. You can think of resource groups as logical containers for the resources you'll create in this tutorial.

In [ ]:
resource_group = "<my-resource-group-name>"

Select the region you wish to deploy your resources in. 

NOTE: You can see a list of the regions under the key 'name' when running the command `az account list-locations`. 

NOTE: Not all regions support GPU enabled VMs. You can check [here](https://azure.microsoft.com/en-us/pricing/details/virtual-machines/linux/)

In [ ]:
region = "<my-selected-region>"

Since we'll be using these settings throughout this tutorial, we'll also same them to the .env file.

In [ ]:
!dotenv set SUBSCRIPTION_ID $subscription_id
!dotenv set RESOURCE_GROUP $resource_group
!dotenv set REGION $region

## Setting up Azure blob storage
In this section of the notebook, we'll create an Azure blob storage that we'll use throughout the tutorial. This object store will be used to store input and output images as well as any supplementary data such as logs and other scripts that will be used in this workflow.

First, create the resource group which we'll put our storage account and all other resources for this project into.

In [ ]:
!az group create -l $region -n $resource_group

Then, create the storage account:
1. Select a name for your storage account and assign it to the variable `storage_account_name`
2. Use the __az cli__ to create the account

In [ ]:
storage_account_name = "<your-storage-acccount-name>"

In [ ]:
!az storage account create -n $storage_account_name -g $resource_group --query 'provisioningState'

Use the __az cli__ to grab the keys of the storage account that was just created. The `--quote '[0].value'` part of the command simply means to select the _value_ of the _zero-th indexed_ of the set of keys.

In [ ]:
key = !az storage account keys list --account-name $storage_account_name -g $resource_group --query '[0].value'

The stdout from the command above is stored in a string array of 1. Select the element in the array and ttrip opening and closing quotation marks.

In [ ]:
storage_account_key = str(key[0][1:-1]) # this is used to strip opening and closing quotation marks

The container in the storage account is a logical container for individual blobs. For this project, we'll store all content into a single container for ease-of-use.
1. Select a name for your storage container and assign it to the variable `azure_container_name`
2. Use the __az cli__ to create the container in the storage account

In [ ]:
azure_container_name = "<name-of-your-storage-container>"

In [ ]:
!az storage container create --account-name $storage_account_name --account-key $storage_account_key --name $azure_container_name

Since we'll be using these settings throughout this tutorial, we'll also same them to the .env file.

In [ ]:
!dotenv set STORAGE_ACCOUNT_NAME $storage_account_name
!dotenv set STORAGE_ACCOUNT_KEY $storage_account_key
!dotenv set AZURE_CONTAINER_NAME $azure_container_name

---

## Conclusion
In this notebook, we've setup a service principal which gives us authentication to create resources in Azure. We've also create a storage account which will use throughout the rest of this tutorial. At this point, we're ready to create our Batch AI cluster and run our work at scale in the cloud.